In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import pickle

data = pd.read_csv('/bohr/train-h5g6/v1/data_train.csv') #Load training set, please do not change this address.读取训练数据，请不要修改这个地址
data = data[~data.shot_made_flag.isna()] 
# Count the number of rows and columns. 分别统计行数和列数
num_rows, num_columns = data.shape
column_names = data.columns
#print(f"行数：{num_rows}")
#print(f"列数：{num_columns}")
print("\n")
#print("列名:")
for col in column_names:
    print(col)


X_train = data[['loc_x', 'loc_y']].values.astype(np.float32) #Load features from training set 读取训练集feature
y_train = data['shot_made_flag'].values.astype(np.float32) #Load label from training set 读取训练集label
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)


# 定义MLP模型 Define the MLP model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        #######请在此处填写代码#######
        #######Please design your model here########
        self.hidden1 = nn.Linear(2, 1)
        self.ac1 = nn.Tanh()
        self.hidden2 = nn.Linear(1, 1)
        self.ac2 = nn.Tanh()
        self.output = nn.Linear(1, 1)
        self.ac3 = nn.Sigmoid()
        
    def forward(self, x):
        #######请在此处填写代码########
        #######Please design your model here########
        x = self.ac1(self.hidden1(x))
        x = self.ac2(self.hidden2(x))
        x = self.ac3(self.output(x))
        return x


def train(model, optimizer, criterion, X_train_tensor, y_train_tensor):
    train_losses = []
    val_losses = []
    num_epochs = 500
    for epoch in range(num_epochs):
        # 训练模式  # Training 
        model.train()
        
        # 前向传播  #Forward calculation
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        
        # 反向传播和优化 #Backward training
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 记录训练损失 #Record the training loss 
        train_losses.append(loss.item())
        
        # 记录训练损失 # Print the training loss
        model.eval()
        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    return train_losses   


model=MyModel()
# 定义损失函数和优化器 # Define the loss function and the optimizer
criterion = nn.BCELoss()  # 二分类最小二乘损失 # BCE Loss
optimizer = optim.Adam(model.parameters(), lr=0.003)  # Adam优化器 # Adam optrimizer
train_losses = train(model, optimizer, criterion, X_train_tensor, y_train_tensor)

In [ ]:
# 请将你的model代码(包括需要import的先导module，例如torch和torch.nn）写在下方以生成一个model的结构文件方便批卷时读取
#Please write your model code (including the necessary preceding modules to import, such as torch and torch.nn) below to generate a model structure file for easy reading during scoring.
model_code = """  
import torch
import torch.nn as nn
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        #######请在此处填写代码#######
        #######Please design your model here########
        self.hidden1 = nn.Linear(2, 1)
        self.ac1 = nn.Tanh()
        self.hidden2 = nn.Linear(1, 1)
        self.ac2 = nn.Tanh()
        self.output = nn.Linear(1, 1)
        self.ac3 = nn.Sigmoid()
        
    def forward(self, x):
        #######请在此处填写代码########
        #######Please design your model here########
        x = self.ac1(self.hidden1(x))
        x = self.ac2(self.hidden2(x))
        x = self.ac3(self.output(x))
        return x
"""
# 将代码写入文件 # Save the model structure
with open('submission_model.py', 'w') as f:
    f.write(model_code)
print("submission_model.py file is generated.")

In [ ]:
# 将模型的参数存储 # Save the model parameter
torch.save(model.state_dict(), 'submission_dic.pth')

In [ ]:
import zipfile
import os

# 定义要打包的文件和压缩文件名 Define the files to be packaged and the compressed file name. 
files_to_zip = ['submission_model.py', 'submission_dic.pth']
zip_filename = 'submission.zip'

# 创建一个 zip 文件, Create a zip file.
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in files_to_zip:
        # 将文件添加到 zip 文件中, Add files to the zip file
        zipf.write(file, os.path.basename(file))

print(f'{zip_filename} is created successfully!')

In [ ]:
!cp submission.zip /personal  
#通过这条指令可以在bohrium的主页中的左侧“文件”中可以该.zip文件并下载
#This instruction allows you to find and download the .zip file in the “Files” section on the left side of the Bohrium homepage.